In [1]:
import pandas as pd

In [2]:
cd ../

/home/anna/PycharmProjects/DataJournalismApp


In [3]:
import util.data_preprocessing as dp
import util.keywords_extraction as ke
import util.topic_extraction as te
import util.ner_finder as nf

# Load and preprocess data

In [5]:
# data = pd.read_csv("data/raw/news33.csv", sep=";")
# data["topic"] = data["rubric"].apply(lambda t: t.split(", ")[-1].split(" ")[0])

# data = pd.read_csv("data/raw/news2020.csv")
# data.drop(["subrubric", "tags"], axis=1, inplace=True)

data = pd.read_csv("data/raw/news_lenta_99-19.csv")

data.head()

,tags,text,title,topic,url
0,Общество,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Мир,https://lenta.ru/news/2018/07/16/su57/
1,Рынки,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Экономика,https://lenta.ru/news/2018/07/16/foes/
2,Преступность,Празднование победы сборной Франции на чемпион...,Празднование победы на ЧМ во Франции закончило...,Мир,https://lenta.ru/news/2018/07/15/fra_bezumie/
3,Политика,География использования лимузинов проекта «Кор...,Песков рассказал о планах на президентские лим...,Россия,https://lenta.ru/news/2018/07/16/delo_shyut/
4,Музыка,Американская поп-певица Бритни Спирс случайно ...,Грудь Бритни Спирс вновь выскочила из лифчика ...,Культура,https://lenta.ru/news/2018/07/16/pedomusk/


In [6]:
# lenta 99-18

data["date"] = pd.to_datetime(data["url"].apply(lambda x: ".".join(x.split("/")[4:7])))
data.drop(["tags", "url"], axis=1, inplace=True)
data = data[data["date"] >= pd.to_datetime("2015.01.01")]
data.reset_index(drop=True, inplace=True)
data.head()

,text,title,topic,date
0,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Мир,2018-07-16
1,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Экономика,2018-07-16
2,Празднование победы сборной Франции на чемпион...,Празднование победы на ЧМ во Франции закончило...,Мир,2018-07-15
3,География использования лимузинов проекта «Кор...,Песков рассказал о планах на президентские лим...,Россия,2018-07-16
4,Американская поп-певица Бритни Спирс случайно ...,Грудь Бритни Спирс вновь выскочила из лифчика ...,Культура,2018-07-16


In [59]:
# news2020

data["source"].unique()

lenta_data = data[data["source"] == 'lenta.ru']
lenta_data["date"] = pd.to_datetime(lenta_data["publication_date"]).values

ria_data = data[data["source"] == 'ria.ru']
ria_data["date"] = pd.to_datetime(ria_data["publication_date"].apply(lambda x: " ".join(x.split(" ")[:2]))).values

name_to_num = {"янв": "01", "фев": "02", "мар": "03", "апр": "04", "мая": "05", "июн": "06", 
               "июл": "07", "авг": "08", "сен": "09", "окт": "10", "ноя": "11", "дек": "12"}
meduza_data = data[data["source"] == 'meduza.io']
meduza_data["date"] = meduza_data["publication_date"].apply(lambda x: x.split(", ")[1].split(" "))
meduza_data["date"] = meduza_data["date"].apply(lambda x: ".".join([x[0], name_to_num[x[1][:3]], x[2]]))


array(['lenta.ru', 'meduza.io', 'ria.ru', 'tjournal.ru'], dtype=object)

In [78]:
tjournal_data = data[data["source"] == 'tjournal.ru']
tjournal_data["_time"] = tjournal_data["publication_date"].astype("int64")
tjournal_data.head()

/home/anna/anaconda3/envs/journalism_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,source,title,text,publication_date,rubric,_time
20803,tjournal.ru,\n ФБК показал филь...,\n\n \n \n \n \n \n...,1598875929,NaN,1598875929
20804,tjournal.ru,\n На улицах Минска...,\n\n \n \n \n \n \n...,1598795150,NaN,1598795150
20805,tjournal.ru,\n В Черногории про...,\n\n \n \n \n \n \n...,1598821788,NaN,1598821788
20806,tjournal.ru,\n Путин и Лукашенк...,\n\n \n \n \n \n \n...,1598784941,NaN,1598784941
20807,tjournal.ru,\n МВД Белоруссии з...,\n\n \n \n \n \n \n...,1598801069,NaN,1598801069


In [76]:
tjournal_data.loc[20803, "text"]

'\n\n    \n    \n        \n        \n        \n             \n    \n         Это первый ролик из фильмов-расследований о региональной политике Сибири. Он посвящён Новосибирску. \n     \n\n        \n        \n            \n                                 \n                                             \n                        \n \n     \n\n         \n\n         \n         104 \n\n        \n             комментария \n\n             \n \n                    \n                     \n                    \n \n\n     \n    \n             В закладки \n    \n     \n \n\n                     \n                                            \n \n          Слушать \n \n                    \n                     \n                    \n \n\n     \n\n        \n             \n\n                 \n             \n\n        \n             \n\n                 \n             \n\n        \n             \n\n                 \n             \n\n        \n             \n\n                 \n             \n\n   

In [7]:
# preprocess text columns and add source and _time cols

# data = dp.preprocess_data(data.drop(["tags"], axis=1), ["title", "text"], "Новости 33", time_col="date")
data = dp.preprocess_data(data, ["title", "text"], "Lenta", time_col="date")

data.head()

,text,title,topic,date,_time,source
0,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Мир,2018-07-16,1531699200,Lenta
1,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Экономика,2018-07-16,1531699200,Lenta
2,Празднование победы сборной Франции на чемпион...,Празднование победы на ЧМ во Франции закончило...,Мир,2018-07-15,1531612800,Lenta
3,География использования лимузинов проекта Корт...,Песков рассказал о планах на президентские лим...,Россия,2018-07-16,1531699200,Lenta
4,Американская поп-певица Бритни Спирс случайно ...,Грудь Бритни Спирс вновь выскочила из лифчика ...,Культура,2018-07-16,1531699200,Lenta


In [8]:
data["art_ind"] = data["date"].astype("str") + ": " + data["title"].apply(lambda x: x.replace("\n", " ").replace("\r", " "))

In [ ]:
# add existing kw and ne to news33 dataset, to save original text format


# kw_ne = pd.read_csv("data/raw_data/news33_kw_ne.csv", index_col=0)
# kw_ne["art_ind"] = kw_ne["date"] + ": " + kw_ne["title"]

# kw_ne.drop("text", axis=1, inplace=True)
# new_data = kw_ne.merge(data[["text", "art_ind"]], on="art_ind", how="inner")
# new_data.fillna("", inplace=True)
# new_data["text"] = new_data["text"].apply(lambda t: t.replace("\n", "\n\r"))
# new_data.to_parquet("news33_processed")

# Find keywords

In [37]:
data_train = data[:10000]
data_test = data[10000:]

In [38]:
data_test = data_test.drop_duplicates(subset="text")

In [44]:
data_test.reset_index(drop=True, inplace=True)
data_test.head()

,text,title,topic,date,_time,source,art_ind
0,"Три россиянина, обучающихся в британских униве...",Среди студентов нашлись желающие вернуться из ...,Россия,2018-04-17,1523923200,Lenta,2018-04-17: Среди студентов нашлись желающие в...
1,Следователи выясняют обстоятельства гибели в С...,Началось разбирательство по поводу гибели русс...,Силовые структуры,2018-04-17,1523923200,Lenta,2018-04-17: Началось разбирательство по поводу...
2,Сайт Роскомнадзора работает с перебоями. Об эт...,Роскомнадзор испытал трудности,Интернет и СМИ,2018-04-17,1523923200,Lenta,2018-04-17: Роскомнадзор испытал трудности
3,Пользователей Pikabu впечатлило креативное ре...,Креативное резюме программиста c лопатой запут...,Интернет и СМИ,2018-04-17,1523923200,Lenta,2018-04-17: Креативное резюме программиста c л...
4,Американский рэпер Кендрик Ламар получил Пулит...,Пулитцеровскую премию впервые в истории присуд...,Культура,2018-04-17,1523923200,Lenta,2018-04-17: Пулитцеровскую премию впервые в ис...


In [11]:
corpus = data_train["text"]
all_text_keywords, vectorizer = ke.get_keywords(corpus)

In [60]:
data_train["keywords"] = all_text_keywords
data_train.head()

,text,title,topic,date,_time,source,art_ind,keywords
0,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Происшествия,2018-07-16,1531699200,Lenta,2018-07-16: Илон Маск назвал педофилом спасавш...,ансворт; затопить; пещера; маск; поглубже; кру...
1,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Общество,2018-07-16,1531699200,Lenta,2018-07-16: США задумались о распечатывании не...,нефть; добыча; венесуэла; баррель; нефтегазовы...
2,Празднование победы сборной Франции на чемпион...,Празднование победы на ЧМ во Франции закончило...,Происшествия,2018-07-15,1531612800,Lenta,2018-07-15: Празднование победы на ЧМ во Франц...,locals; франция; порядок; беспорядки; город; у...
3,География использования лимузинов проекта Корт...,Песков рассказал о планах на президентские лим...,Власть\nКультура,2018-07-16,1531699200,Lenta,2018-07-16: Песков рассказал о планах на прези...,кортеж; лимузин; автомобиль; проект; mercedes;...
4,Американская поп-певица Бритни Спирс случайно ...,Грудь Бритни Спирс вновь выскочила из лифчика ...,Власть,2018-07-16,1531699200,Lenta,2018-07-16: Грудь Бритни Спирс вновь выскочила...,спирс; оксон; бритни; выступление; бюстгальтер...


In [47]:
test_kw = []
for i in range(len(data_test) // 10000 + 1):
    test_text_keywords = ke.get_keywords_preprocessed(data_test.loc[i*10000:(i+1)*10000-1, "text"].values, vectorizer)
    test_kw.extend(list(zip(data_test.loc[i*10000:(i+1)*10000-1, "art_ind"].values, test_text_keywords)))
    print(str(i) + " done")
    print(test_kw[-10:])
data_test["keywords"] = test_kw
data_test.head()

0 done
[('2018-01-20: Мутко назвал число российских участников Олимпиады', 'олимпийский; спортсмены; окр; мутко; спортсменов; смочь; вице; принять; атлеты; полагаем'), ('2018-01-20: Вера Брежнева станцевала на\xa0парковке и\xa0смутила американских водителей', 'брежнев; певица; худшее; стс; киркорова; дурачиться; гнойного; водителями; коронный; ролик'), ('2018-01-20: Молдавия предъявит России миллиардный счет за\xa0Приднестровье', 'приднестровье; молдавский; пмр; присутствие; молдавия; приднестровский; российский; молдавии; военный; международный'), ('2018-01-19: Променявшая Чертаново на\xa0Испанию футболистка пожаловалась на\xa0одиночество', 'чертаново; футболистка; испании; бывает; спортсменка; языке; пальцах; втроем; никого; трудом'), ('2018-01-19: На фото Бузовой обнаружили НЛО', 'бузов; объект; странный; летать; задний; подписчик; засомневаться; фото; снимка; барбекю'), ('2018-01-19: Сын бывшего премьера возглавит главный военный банк России', 'банк; фрадков; промсвязьбанк; банка; 

5 done
[('2017-02-25: В Германии водитель въехал в\xa0группу пешеходов', 'мужчина; наезда; одиночку; злоумышленника; тяжелое; организациям; focus; пешеходов; наезд; полиция'), ('2017-02-25: Опубликовано видео визита Савченко к\xa0пленным силовикам в\xa0ДНР', 'савченко; массировать; обстрел; украинский; визит; донецкий; силовиками; пленных; исключаем; агломерация'), ('2017-02-25: В Сирийской свободной армии назвали условия сотрудничества с\xa0Москвой', 'сирийский; вкп; усман; оппозиции; группа; переговоры; эль; оппозиционный; иг; режим'), ('2017-02-25: В Горбачев-фонде опровергли наличие лондонского архива', 'горбачев; фонда; особняк; поляков; фонд; архив; захватить; помещение; офис; организация'), ('2017-02-25: Устюгов прокомментировал первое золото ЧМ\xa0в карьере', 'устюгов; секунды; дистанция; мартин; понял; преодолеть; победа; выиграть; спринте; многоднёвка'), ('2017-02-25: На Кубе столкнулись два поезда', 'пассажирский; грузовой; железнодорожный; авария; тростник; критическое; рез

10 done
[('2016-05-12: Таможенники задержали в\xa0Москве партию ценностей на\xa012\xa0миллиардов рублей', 'таможенный; фтс; товарами; ввезти; миллиард; рублей; служба; ценность; москвы; доход'), ('2016-05-12: Порошенко разрешил назначать генпрокурора без высшего юридического образования', 'генпрокурор; должность; юридический; отставка; стаж; порошенко; украина; образования; работа; законодательный'), ('2016-05-12: Руперт Мердок оказался теневым совладельцем Uber', 'uber; мердок; такси; стартап; руперт; оцениваться; долларов; совладелец; владеть; бизнесмен'), ('2016-05-12: Бывший игрок Локомотива во\xa0время матча спас жизнь японскому футболисту', 'ванкувер; чикаго; уайткэпс; локомотив; суметь; румын; mls; вытекать; рта; запасть'), ('2016-05-12: В Совфеде выступили против переноса российской столицы', 'матвиенко; перенос; вопрос; столицы; председатель; севастополь; спикер; брянской; столица; этота'), ('2016-05-12: В Совфеде пригрозили сохранением ядерного потенциала из-за комплексов ПРО

15 done
[('2015-04-23: Российские самолеты вывезли из\xa0Йемена еще 300\xa0человек', 'йемен; узбекистана; хади; аэродром; покинуть; гражданин; страну; борт; граждан; чкаловский'), ('2015-04-23: В Открытом правительстве проверили реакцию министерств на\xa0коррупцию', 'факт; коррупции; коррупция; сообщение; минкультуры; телефона; ведомство; связаться; наличие; поиск'), ('2015-04-23: Керри оправдал присутствие американских военных на\xa0Украине', 'украины; восток; керри; инструктор; ополченец; украинский; военный; боев; донбасса; госдепартамент'), ('2015-04-23: Минфин предложил оптимизировать расходы на\xa0образование и\xa0медицину', 'бюджетный; бюджетник; сокращение; минфин; процентов; учреждения; документ; государственный; имущество; сектор'), ('2015-04-23: Белорусу дали два года за\xa0голое селфи', 'барановичский; баран; селфи; житель; суд; intex; снисхождение; межрайонный; интернет; признать'), ('2015-04-23: Правительство сохранило накопительную часть пенсии', 'накопительный; пенсионн

,text,title,topic,date,_time,source,art_ind,keywords
0,"Три россиянина, обучающихся в британских униве...",Среди студентов нашлись желающие вернуться из ...,Россия,2018-04-17,1523923200,Lenta,2018-04-17: Среди студентов нашлись желающие в...,(2018-04-17: Среди студентов нашлись желающие ...
1,Следователи выясняют обстоятельства гибели в С...,Началось разбирательство по поводу гибели русс...,Силовые структуры,2018-04-17,1523923200,Lenta,2018-04-17: Началось разбирательство по поводу...,(2018-04-17: Началось разбирательство по повод...
2,Сайт Роскомнадзора работает с перебоями. Об эт...,Роскомнадзор испытал трудности,Интернет и СМИ,2018-04-17,1523923200,Lenta,2018-04-17: Роскомнадзор испытал трудности,"(2018-04-17: Роскомнадзор испытал трудности, п..."
3,Пользователей Pikabu впечатлило креативное ре...,Креативное резюме программиста c лопатой запут...,Интернет и СМИ,2018-04-17,1523923200,Lenta,2018-04-17: Креативное резюме программиста c л...,(2018-04-17: Креативное резюме программиста c ...
4,Американский рэпер Кендрик Ламар получил Пулит...,Пулитцеровскую премию впервые в истории присуд...,Культура,2018-04-17,1523923200,Lenta,2018-04-17: Пулитцеровскую премию впервые в ис...,(2018-04-17: Пулитцеровскую премию впервые в и...


In [61]:
test_kw = [x[1] for x in test_kw]
data_test["keywords"] = test_kw
data_test.head()

,text,title,topic,date,_time,source,art_ind,keywords
0,"Три россиянина, обучающихся в британских униве...",Среди студентов нашлись желающие вернуться из ...,Россия,2018-04-17,1523923200,Lenta,2018-04-17: Среди студентов нашлись желающие в...,сардарян; мгимо; недружественный; вуз; студент...
1,Следователи выясняют обстоятельства гибели в С...,Началось разбирательство по поводу гибели русс...,Силовые структуры,2018-04-17,1523923200,Lenta,2018-04-17: Началось разбирательство по поводу...,пшеничный; пшеничного; фсин; сизый; смерть; ва...
2,Сайт Роскомнадзора работает с перебоями. Об эт...,Роскомнадзор испытал трудности,Интернет и СМИ,2018-04-17,1523923200,Lenta,2018-04-17: Роскомнадзор испытал трудности,перебоями; роскомнадзор; сайт; неполадка; недо...
3,Пользователей Pikabu впечатлило креативное ре...,Креативное резюме программиста c лопатой запут...,Интернет и СМИ,2018-04-17,1523923200,Lenta,2018-04-17: Креативное резюме программиста c л...,резюме; 50; чело; находчивый; расстраивать; по...
4,Американский рэпер Кендрик Ламар получил Пулит...,Пулитцеровскую премию впервые в истории присуд...,Культура,2018-04-17,1523923200,Lenta,2018-04-17: Пулитцеровскую премию впервые в ис...,ламар; пулитцеровский; музыкальный; the; рэпер...


In [65]:
data = pd.concat([data_train, data_test])
data = data.reset_index(drop=True)
data.head()

,text,title,topic,date,_time,source,art_ind,keywords
0,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Происшествия,2018-07-16,1531699200,Lenta,2018-07-16: Илон Маск назвал педофилом спасавш...,ансворт; затопить; пещера; маск; поглубже; кру...
1,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Общество,2018-07-16,1531699200,Lenta,2018-07-16: США задумались о распечатывании не...,нефть; добыча; венесуэла; баррель; нефтегазовы...
2,Празднование победы сборной Франции на чемпион...,Празднование победы на ЧМ во Франции закончило...,Происшествия,2018-07-15,1531612800,Lenta,2018-07-15: Празднование победы на ЧМ во Франц...,locals; франция; порядок; беспорядки; город; у...
3,География использования лимузинов проекта Корт...,Песков рассказал о планах на президентские лим...,Власть\nКультура,2018-07-16,1531699200,Lenta,2018-07-16: Песков рассказал о планах на прези...,кортеж; лимузин; автомобиль; проект; mercedes;...
4,Американская поп-певица Бритни Спирс случайно ...,Грудь Бритни Спирс вновь выскочила из лифчика ...,Власть,2018-07-16,1531699200,Lenta,2018-07-16: Грудь Бритни Спирс вновь выскочила...,спирс; оксон; бритни; выступление; бюстгальтер...


In [66]:
data.to_csv("data/processed/lenta_kw.csv")

# Find topics

In [52]:
topic_keywords, topics_pred, vectorizer, lda = te.get_topics(data_train["text"].values, topics=25, n_keywords=10)

In [53]:
topic_keywords

{0: ['ребёнок',
  'школа',
  'женщина',
  'девочка',
  'свой',
  'родитель',
  'мальчик',
  'летний',
  'стать',
  'детский'],
 1: ['город',
  'животное',
  'обнаружить',
  'сообщать',
  'штат',
  'животный',
  'из',
  'за',
  'найти',
  'издание'],
 2: ['премьер',
  'армения',
  'пост',
  'партия',
  'министр',
  'сообщать',
  'саргсян',
  'протест',
  'апрель',
  'пашинянин'],
 3: ['пользователь',
  'свой',
  'опубликовать',
  'видео',
  'девушка',
  'фотография',
  'twitter',
  'тысяча',
  'ролик',
  'сеть'],
 4: ['чемпионат',
  'матч',
  'команда',
  'сборный',
  'мир',
  'россия',
  'сборная',
  'счёт',
  'мира',
  'стать'],
 5: ['tesla',
  'миллион',
  'долларов',
  'свой',
  'маск',
  'тысяча',
  'илона',
  'доллар',
  'компания',
  'самолёт'],
 6: ['бабченко',
  'свой',
  'сбу',
  'журналист',
  'убийство',
  'российский',
  'боец',
  'служба',
  'украина',
  'бой'],
 7: ['мужчина',
  'летний',
  'сообщать',
  'полиция',
  'произойти',
  'инцидент',
  'сотрудник',
  'город',
  

In [67]:
topic_list = """Общество Общество Власть Общество Спорт Общество Происшествия Происшествия Власть Происшествия Власть Культура Новости Культура Власть Власть Власть Новости Происшествия Происшествия Власть Общество Культура Происшествия Власть"""
topics = dict(zip(list(range(25)), topic_list.split(" ")))

In [71]:
test_topics = []
for i in range(len(data_test) // 10000 + 1):
    test_topics_pred = te.get_topics_pretrained(data_test.loc[i*10000:(i+1)*10000-1, "text"].values, lda, vectorizer)
    test_topics.extend(list(zip(data_test.loc[i*10000:(i+1)*10000-1, "art_ind"].values, test_topics_pred)))
    print(str(i) + " done")
    print(test_topics[-10:])
# data_test["keywords"] = test_kw
# data_test.head()

0 done
[('2018-01-20: Мутко назвал число российских участников Олимпиады', 14), ('2018-01-20: Вера Брежнева станцевала на\xa0парковке и\xa0смутила американских водителей', 3), ('2018-01-20: Молдавия предъявит России миллиардный счет за\xa0Приднестровье', 8), ('2018-01-19: Променявшая Чертаново на\xa0Испанию футболистка пожаловалась на\xa0одиночество', 4), ('2018-01-19: На фото Бузовой обнаружили НЛО', 3), ('2018-01-19: Сын бывшего премьера возглавит главный военный банк России', 20), ('2018-01-19: В США отменили запуск ракеты Atlas V\xa0с военным спутником', 24), ('2018-01-19: Папа Римский поддержал педофилов и\xa0взбесил чилийцев', 11), ('2018-01-19: Глумившуюся над больными медсестру на\xa0Сахалине уволили за\xa0разглашение тайны', 3), ('2018-01-18: Появилось видео виртуозной посадки самолета в\xa0Дюссельдорфе во\xa0время урагана', 1)]
1 done
[('2017-10-24: Путин испытал гордость за\xa0людей труда', 4), ('2017-10-24: Разработчики поделились планами по\xa0созданию улучшенного биткоина

10 done
[('2016-05-12: Таможенники задержали в\xa0Москве партию ценностей на\xa012\xa0миллиардов рублей', 20), ('2016-05-12: Порошенко разрешил назначать генпрокурора без высшего юридического образования', 8), ('2016-05-12: Руперт Мердок оказался теневым совладельцем Uber', 20), ('2016-05-12: Бывший игрок Локомотива во\xa0время матча спас жизнь японскому футболисту', 4), ('2016-05-12: В Совфеде выступили против переноса российской столицы', 8), ('2016-05-12: В Совфеде пригрозили сохранением ядерного потенциала из-за комплексов ПРО США', 24), ('2016-05-12: Власти Украины придумали способ борьбы с\xa0российскими книгами', 8), ('2016-05-12: Пользовательнице Twitter предложили купить марихуану в\xa0полицейском участке', 3), ('2016-05-12: В норвежском городе запретили использовать в\xa0рекламе образы голых людей', 19), ('2016-05-12: ФСБ приобретет два самолета австрийской разработки для охраны границ', 24)]
11 done
[('2016-03-14: В водопроводе американской больницы нашли неизвестные бактери

In [72]:
test_topics = [x[1] for x in test_topics]

In [75]:
data_test = te.set_topics_to_data(data_test, test_topics, topics)
data_test.head()

,text,title,topic,date,_time,source,art_ind,keywords
0,"Три россиянина, обучающихся в британских униве...",Среди студентов нашлись желающие вернуться из ...,Новости,2018-04-17,1523923200,Lenta,2018-04-17: Среди студентов нашлись желающие в...,сардарян; мгимо; недружественный; вуз; студент...
1,Следователи выясняют обстоятельства гибели в С...,Началось разбирательство по поводу гибели русс...,Происшествия,2018-04-17,1523923200,Lenta,2018-04-17: Началось разбирательство по поводу...,пшеничный; пшеничного; фсин; сизый; смерть; ва...
2,Сайт Роскомнадзора работает с перебоями. Об эт...,Роскомнадзор испытал трудности,Общество,2018-04-17,1523923200,Lenta,2018-04-17: Роскомнадзор испытал трудности,перебоями; роскомнадзор; сайт; неполадка; недо...
3,Пользователей Pikabu впечатлило креативное ре...,Креативное резюме программиста c лопатой запут...,Общество,2018-04-17,1523923200,Lenta,2018-04-17: Креативное резюме программиста c л...,резюме; 50; чело; находчивый; расстраивать; по...
4,Американский рэпер Кендрик Ламар получил Пулит...,Пулитцеровскую премию впервые в истории присуд...,Культура,2018-04-17,1523923200,Lenta,2018-04-17: Пулитцеровскую премию впервые в ис...,ламар; пулитцеровский; музыкальный; the; рэпер...


In [76]:
data = pd.concat([data_train, data_test])
data = data.reset_index(drop=True)
data.head()

,text,title,topic,date,_time,source,art_ind,keywords
0,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Происшествия,2018-07-16,1531699200,Lenta,2018-07-16: Илон Маск назвал педофилом спасавш...,ансворт; затопить; пещера; маск; поглубже; кру...
1,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Власть,2018-07-16,1531699200,Lenta,2018-07-16: США задумались о распечатывании не...,нефть; добыча; венесуэла; баррель; нефтегазовы...
2,Празднование победы сборной Франции на чемпион...,Празднование победы на ЧМ во Франции закончило...,Происшествия,2018-07-15,1531612800,Lenta,2018-07-15: Празднование победы на ЧМ во Франц...,locals; франция; порядок; беспорядки; город; у...
3,География использования лимузинов проекта Корт...,Песков рассказал о планах на президентские лим...,Власть,2018-07-16,1531699200,Lenta,2018-07-16: Песков рассказал о планах на прези...,кортеж; лимузин; автомобиль; проект; mercedes;...
4,Американская поп-певица Бритни Спирс случайно ...,Грудь Бритни Спирс вновь выскочила из лифчика ...,Культура,2018-07-16,1531699200,Lenta,2018-07-16: Грудь Бритни Спирс вновь выскочила...,спирс; оксон; бритни; выступление; бюстгальтер...


In [81]:
data.to_csv("data/processed/lenta_topic_kw.csv")

In [4]:
data = pd.read_csv("data/processed/lenta_topic_kw.csv", index_col=0)
data.head()

,text,title,topic,date,_time,source,art_ind,keywords
0,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Происшествия,2018-07-16,1531699200,Lenta,2018-07-16: Илон Маск назвал педофилом спасавш...,ансворт; затопить; пещера; маск; поглубже; кру...
1,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Власть,2018-07-16,1531699200,Lenta,2018-07-16: США задумались о распечатывании не...,нефть; добыча; венесуэла; баррель; нефтегазовы...
2,Празднование победы сборной Франции на чемпион...,Празднование победы на ЧМ во Франции закончило...,Происшествия,2018-07-15,1531612800,Lenta,2018-07-15: Празднование победы на ЧМ во Франц...,locals; франция; порядок; беспорядки; город; у...
3,География использования лимузинов проекта Корт...,Песков рассказал о планах на президентские лим...,Власть,2018-07-16,1531699200,Lenta,2018-07-16: Песков рассказал о планах на прези...,кортеж; лимузин; автомобиль; проект; mercedes;...
4,Американская поп-певица Бритни Спирс случайно ...,Грудь Бритни Спирс вновь выскочила из лифчика ...,Культура,2018-07-16,1531699200,Lenta,2018-07-16: Грудь Бритни Спирс вновь выскочила...,спирс; оксон; бритни; выступление; бюстгальтер...


In [10]:
len(data)

183396

In [8]:
data[data["art_ind"] == "2018-03-29: Минфин припугнул Великобританию"]

,text,title,topic,date,_time,source,art_ind,keywords
12006,"Замминистра финансов Алексей Моисеев заявил, ч...",Минфин припугнул Великобританию,Власть,2018-03-29,1522281600,Lenta,2018-03-29: Минфин припугнул Великобританию,бумага; евробонд; моисеев; инвесторов; долгов;...


In [11]:
data[12007:].to_csv("data/processed/lenta_ner_raw.csv")

In [6]:
ner_data = pd.read_csv("data/processed/lenta_ner_part2.csv", index_col=0)

In [7]:
ner_data

,entity
art_ind,
2018-06-21: Цена литра бензина пробила психологическую отметку,Кубани
2018-06-21: Цена литра бензина пробила психологическую отметку,Об
2018-06-21: Цена литра бензина пробила психологическую отметку,Росстата
2018-06-21: Цена литра бензина пробила психологическую отметку,Краснодарскому краю
2018-06-21: Цена литра бензина пробила психологическую отметку,Так
...,...
2018-03-29: Минфин припугнул Великобританию,Россией
2018-03-29: Минфин припугнул Великобританию,Лондона
2018-03-29: Минфин припугнул Великобританию,Как


# Find NE

In [36]:
data["ner"] = data["text"].apply(nf.finder)

In [ ]:
all_ner = []
for i in range(len(data_test) // 10000 + 1):
    all_ner.extend([nf.finder(text) for text in data.loc[i*10000:(i+1)*10000-1, "text"].values])

In [37]:
data["ner"] = data["ner"].apply(lambda x: "; ".join([w.lower() for w in x]))
data.head()

,date,rubric,title,text,_time,source,keywords,topic,ner
index,,,,,,,,,
0,2019-08-17,Новости 33; Общество,Прокуратура обнаружила ФАПы-призраки в Суздаль...,"Прокуратура обнаружила, что паспорт региональн...",1566000000,news33,фапах; фап; медицинский; пункт; район; деятель...,август,суздальский; большое борисово; фапы район; лем...
1,2019-08-17,Новости 33; Происшествия 33,В Гусь-Хрустальном женщина перевела мошенникам...,Следственный отдел полиции в Гусь-Хрустальном ...,1566000000,news33,женщина; хрустальный; гусь; потерпевший; звони...,дтп,гусь-хрустальный; россия; мо; мвд; вгусе.ру; с...
2,2019-08-16,Новости 33; Общество,Жириновский: Мое имя тоже - Владимир. Всё сошл...,Во Владимирском академическом театре драмы про...,1565913600,news33,жириновский; губернатор; сипягин; собрание; лд...,социум,владимирская область; россия; цфо; владимирски...
3,2019-08-16,Новости 33; Общество,Во Владимире трубопровод на улице Герцена убер...,Во Владимире модернизируют тепловую сеть на ул...,1565913600,news33,теплов; составить; сетей; отопительный; м²; ра...,социум,во владимир; герцен; т плюс; владимирская тэц-2
4,2019-08-16,Новости 33; Общество,Выпускник ВлГУ приблизил эру квантовых компьют...,Выпускник кафедры физики и прикладной математи...,1565913600,news33,скорость; физика; света; поток; полупроводнико...,убийство,влгу; евгений седов
